In [66]:
# Necessary imports ... 
import matplotlib.pyplot as plt
from astropy.table import Table
import os
import numpy as np

In [247]:
#  Read in the DECam - MAF matched data ... 
percent = 0.51  # or 0.06  0.21   0.51
fname = 'table_decam_maf_matched_'+str(percent)+'_density_percent.vo.xml'
decam_maf = Table.read(fname)
print('We read the DECam - MAF matches within 30 arcsec from: \n%s'%fname)
# Check which files are available in the archive dir 
outDir = '../raw_data/DECam/'
archive_files = np.array(decam_maf['archive_file'].data).astype(str)
print('\nWe have DECam data for %s  percent matched to MAF for : '%str(percent))
print(np.array(os.listdir(outDir))[mask])

# choose available fields ....
# since only on those we can run DAOPhot, 
# and it makes sense to get TRILEGAL data 
sampled_objects = decam_maf[np.in1d(archive_files, os.listdir(outDir))]




We read the DECam - MAF matches within 30 arcsec from: 
table_decam_maf_matched_0.51_density_percent.vo.xml

We have DECam data for 0.51  percent matched to MAF for : 
['tu1669764.fits.fz' 'tu1670143.fits.fz' 'tu1677011.fits.fz'
 'tu2091292.fits.fz' 'tu2093190.fits.fz']


### Run TRILEGAL queries in a loop

In [248]:
from astropy.io import fits
import re
from mechanize import Browser

def trilegal_counts(fname):
    hdu = fits.open(outDir+fname)
    print('\nInvestigated DECam file is %s'%fname)
    print('DECam ra,dec = %f, %f'%(sampled_objects['ra'][j],sampled_objects['dec'][j] ))
    MAF_count = sampled_objects['MAF_px_density'][j]
    print('MAF count here is %d'%MAF_count)

    x_px_scale = hdu[0].header['PIXSCAL1']  # arcsec / pixel
    y_px_scale = hdu[0].header['PIXSCAL2']
    x_n_pixels = hdu[1].header['NAXIS1']
    y_n_pixels = hdu[1].header['NAXIS2']
    x_arcsec = x_n_pixels * x_px_scale
    y_arcsec = y_n_pixels * y_px_scale
    print('Image size in arcmin is %f by %f'%(x_arcsec / 60, y_arcsec / 60 ))
    area_size_sq_arcsec = x_arcsec * y_arcsec
    area_size_sq_deg = area_size_sq_arcsec / (3600 * 3600)
    print('Single image frame area is %d sq. arcsec, i.e. %f sq.deg.' %(area_size_sq_arcsec, 
                                                        area_size_sq_deg ))

    # grab ra, dec ...
    ra = sampled_objects[j]['ra']
    dec = sampled_objects[j]['dec']
    archive_file = sampled_objects[j]['archive_file']
    print('Obtaining TRILEGAL data for object %s, at \
    \nDECam ra,dec = %s,%s...'%(archive_file,ra,dec))

    # mechanize : fill in the TRILEGAL form with 
    # an automated python form 
    # this works only with python 2.7 kernel .. 

    print('Querying TRILEGAL for object at given equatorial coordinates')
    print('Within the given area, cutting on r<24.5')
    browser = Browser()
    browser.open("http://stev.oapd.inaf.it/cgi-bin/trilegal")
    browser.select_form(nr=0) # select the first  (and only) available form...
    browser['gal_coord'] = ["2"]  # select galactic coords 
    ra_hrs = (ra/360) * 24.0 # convert degrees to hours 
    browser['eq_alpha'] = str(ra_hrs)  # in hours 
    browser['eq_delta'] = str(dec)  # in degrees 
    browser['field'] = str(area_size_sq_deg)
    photsys_file = "tab_mag_odfnew/tab_mag_lsst.dat"
    browser['photsys_file'] = [photsys_file]
    icm_lim = "3" ; mag_lim = "24.5"
    browser["icm_lim"] = icm_lim
    browser["mag_lim"] = mag_lim                               
    response = browser.submit()
    #content = response.read()
    # print out links to find out 
    # which is the one of interest
    #for link in browser.links():
    #    print link.text, link.url

    print('Asserting that TRILEGAL normally terminated...')
    lastline = ""
    while lastline != '#TRILEGAL normally terminated\n' : 
        # wait for 30 sec...  
        print('Waiting for 30 sec more ...')
        time.sleep(30)
        
        # try downloading the result... 
        url = 'http://stev.oapd.inaf.it/' + str(browser.links()[0].url[3:])
        filename = 'trilegal_'+str(key)+ '_j-'+str(j)+'.dat'
        browser.retrieve(url, filename)

        # only exit the loop when lastline is 
        # a correct termination
        fi=open(filename, 'r')
        lastline=""
        for line in fi:
            lastline = line

    # then read the table and count 
    trilegal_data = Table.read(filename, format='ascii')
    f_length = len(trilegal_data)
    n_per_sq_deg = f_length / area_size_sq_deg
    print('There are %d stars in that field , corresponding \
    to %d per square degree '%(f_length,n_per_sq_deg))
    print('\n')
    
    results = {'archive_file':fname , 'ra_deg':ra,'ra_hrs':ra_hrs, 'dec_deg':dec,
              'field':area_size_sq_deg, 'photsys_file':photsys_file,
              'icm_lim':icm_lim, 'mag_lim':mag_lim, 'filename':filename,
              'sources':f_length, 'source_density':n_per_sq_deg, 'MAF_count':MAF_count
              } 
    return results

In [250]:

# run TRILEGAL  queries in the loop, and store results .. 

res_dic = {'archive_file':[] , 'ra_deg':[],'ra_hrs':[], 'dec_deg':[],
              'field':[], 'photsys_file':[],
              'icm_lim':[], 'mag_lim':[], 'filename':[],
              'sources':[], 'source_density':[],'MAF_count':[]
          }
           
for j in range(len(sampled_objects)):
    fname = np.array(sampled_objects['archive_file']).astype(str)[j]
    results = trilegal_counts(fname)
    for key in results.keys():
        res_dic[key].append(results[key])
        
#Save the results of counting...
fname = 'TRILEGAL_'+str(percent)+'_perc_DECam_MAF_counts.csv'
Table(res_dic).write(fname, overwrite='True')
print('Results of TRILEGAL queries are saved as %s'%fname)


Investigated DECam file is c4d_160607_025052_ooi_g_v1.fits.fz
DECam ra,dec = 233.175667, -1.679167
MAF count here is 20052
Image size in arcmin is 9.207000 by 18.423000
Single image frame area is 610634 sq. arcsec, i.e. 0.047117 sq.deg.
Obtaining TRILEGAL data for object c4d_160607_025052_ooi_g_v1.fits.fz, at     
DECam ra,dec = 233.175667,-1.679167...
Querying TRILEGAL for object at given equatorial coordinates
Within the given area, cutting on r<24.5
Asserting that TRILEGAL normally terminated...
Waiting for 30 sec more ...
Waiting for 30 sec more ...
There are 1302 stars in that field , corresponding     to 27633 per square degree 



Investigated DECam file is c4d_170718_042438_ooi_r_v1.fits.fz
DECam ra,dec = 228.030375, -6.490833
MAF count here is 20412
Image size in arcmin is 9.207000 by 18.423000
Single image frame area is 610634 sq. arcsec, i.e. 0.047117 sq.deg.
Obtaining TRILEGAL data for object c4d_170718_042438_ooi_r_v1.fits.fz, at     
DECam ra,dec = 228.030375,-6.490833..

In [251]:
# Display the results...
print('percent = %f'%percent)
trilegal = Table.read('TRILEGAL_'+str(percent)+'_perc_DECam_MAF_counts.csv')
trilegal

percent = 0.510000


icm_lim,MAF_count,mag_lim,ra_deg,sources,dec_deg,ra_hrs,source_density,archive_file,filename,field,photsys_file
int64,float64,float64,float64,int64,float64,float64,float64,str34,str29,float64,str31
3,20052.0,24.5,233.175667,1302,-1.679167,15.5450444667,27633.4423867,c4d_160607_025052_ooi_g_v1.fits.fz,trilegal_photsys_file_j-0.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat
3,20412.0,24.5,228.030375,1273,-6.490833,15.202025,27017.9509665,c4d_170718_042438_ooi_r_v1.fits.fz,trilegal_photsys_file_j-1.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat
3,21024.0,24.5,223.138875,1256,-11.249944,14.875925,26657.1456511,c4d_150615_005257_ooi_g_v1.fits.fz,trilegal_photsys_file_j-2.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat
3,20916.0,24.5,222.213375,1321,-12.759667,14.814225,28036.6953862,c4d_150614_002629_ooi_g_v1.fits.fz,trilegal_photsys_file_j-3.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat
3,19944.0,24.5,91.8075,1948,-14.163028,6.1205,41344.0443697,tu2046406.fits.fz,trilegal_photsys_file_j-4.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat


## Combine DAOStarFinder and TRILEGAL counts

In [166]:
from astropy.table import join

In [252]:

print('precent = %f'%percent)
trilegal = Table.read('TRILEGAL_'+str(percent)+'_perc_DECam_MAF_counts.csv')
dao = Table.read('DAO_'+str(percent)+'_perc_DECam_MAF_counts.csv')
merged1= join(trilegal,dao, keys='archive_file',table_names=['TRI', 'DAO'])
merged1

precent = 0.510000


icm_lim,MAF_count,mag_lim,ra_deg,sources_TRI,dec_deg,ra_hrs,source_density_TRI,archive_file,filename,field,photsys_file,area_sq_deg,fwhm,threshold,sources_DAO,source_density_DAO
int64,float64,float64,float64,int64,float64,float64,float64,str34,str29,float64,str31,float64,float64,float64,int64,float64
3,20916.0,24.5,222.213375,1321,-12.759667,14.814225,28036.6953862,c4d_150614_002629_ooi_g_v1.fits.fz,trilegal_photsys_file_j-3.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat,0.0471168225,8.060747,31.453033182,366,7767.92620088
3,21024.0,24.5,223.138875,1256,-11.249944,14.875925,26657.1456511,c4d_150615_005257_ooi_g_v1.fits.fz,trilegal_photsys_file_j-2.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat,0.0471168225,5.475378,31.0363521064,608,12904.095984
3,20052.0,24.5,233.175667,1302,-1.679167,15.5450444667,27633.4423867,c4d_160607_025052_ooi_g_v1.fits.fz,trilegal_photsys_file_j-0.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat,0.0471168225,6.117483,31.5070427428,630,13371.0205097
3,20412.0,24.5,228.030375,1273,-6.490833,15.202025,27017.9509665,c4d_170718_042438_ooi_r_v1.fits.fz,trilegal_photsys_file_j-1.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat,0.0471168225,5.847024,54.0614157019,830,17615.7889255
3,19944.0,24.5,91.8075,1948,-14.163028,6.1205,41344.0443697,tu2046406.fits.fz,trilegal_photsys_file_j-4.dat,0.0471168225,tab_mag_odfnew/tab_mag_lsst.dat,0.0471168225,4.202916,55.1179480827,1695,35974.4123238


In [253]:
# only here need to add MAF counts 
# nbecause I forgot to add them in TRILEGAL 
# loop  -  later on this should be deleted.
sampled_objects['archive_file'] = sampled_objects['archive_file'].astype(str)
merged = join(merged1,  sampled_objects[['archive_file','MAF_px_density']])

merged.rename_column('archive_file', 'archive')
merged.rename_column('ra_deg','ra')
merged.rename_column('dec_deg','dec')
merged.rename_column('source_density_TRI','TRILEGAL')
merged.rename_column('MAF_px_density','MAF')
merged.rename_column('source_density_DAO','DAO')

for col in ['TRILEGAL', 'DAO', 'MAF']:
    merged[col] = merged[col].astype(int)

In [254]:
merged[['archive','ra','dec', 'TRILEGAL', 'MAF', 'DAO' ]]

archive,ra,dec,TRILEGAL,MAF,DAO
str34,float64,float64,int64,int64,int64
c4d_150614_002629_ooi_g_v1.fits.fz,222.213375,-12.759667,28036,20916,7767
c4d_150615_005257_ooi_g_v1.fits.fz,223.138875,-11.249944,26657,21024,12904
c4d_160607_025052_ooi_g_v1.fits.fz,233.175667,-1.679167,27633,20052,13371
c4d_170718_042438_ooi_r_v1.fits.fz,228.030375,-6.490833,27017,20412,17615
tu2046406.fits.fz,91.8075,-14.163028,41344,19944,35974


In [255]:
# Save the info : 
# archive_name   |  ra_deg  | dec_deg | MAF count  | DAO count | TRILEGAL count 

# Save as LaTeX file so that I can quickly share with Colin
from astropy.io import ascii
ascii.write(merged[['archive','ra','dec', 'TRILEGAL', 'MAF', 'DAO' ]], format='latex')  

\begin{table}
\begin{tabular}{cccccc}
archive & ra & dec & TRILEGAL & MAF & DAO \\
c4d_150614_002629_ooi_g_v1.fits.fz & 222.213375 & -12.759667 & 28036 & 20916 & 7767 \\
c4d_150615_005257_ooi_g_v1.fits.fz & 223.138875 & -11.249944 & 26657 & 21024 & 12904 \\
c4d_160607_025052_ooi_g_v1.fits.fz & 233.175667 & -1.679167 & 27633 & 20052 & 13371 \\
c4d_170718_042438_ooi_r_v1.fits.fz & 228.030375 & -6.490833 & 27017 & 20412 & 17615 \\
tu2046406.fits.fz & 91.8075 & -14.163028 & 41344 & 19944 & 35974 \\
\end{tabular}
\end{table}


archive,ra,dec,TRILEGAL,DAO,MAF
str17,float64,float64,float64,float64,float64
tu1669764.fits.fz,260.973083,-18.467306,1545804.68904,376915.594089,641484.0
tu1670143.fits.fz,259.304375,-19.487611,1229290.03354,387454.634903,599508.0
tu1677011.fits.fz,260.957167,-20.493167,2533705.61261,509093.625318,810144.0
tu2091292.fits.fz,262.64325,-19.471278,3357866.50299,434409.599671,795996.0
tu2093190.fits.fz,260.994583,-16.475528,1036105.52261,286712.882644,585576.0


\begin{table}
\begin{tabular}{cccccccccccccccc}
icm_lim & mag_lim & ra_deg & sources_TRI & dec_deg & ra_hrs & source_density_TRI & archive_file & filename & field & photsys_file & area_sq_deg & fwhm & threshold & sources_DAO & source_density_DAO \\
3 & 24.5 & 260.973083 & 72017 & -18.467306 & 17.3982055333 & 1545804.68904 & tu1669764.fits.fz & trilegal_photsys_file_j-1.dat & 0.04658868 & tab_mag_odfnew/tab_mag_lsst.dat & 0.04658868 & 4.877344 & 144.051478584 & 17560 & 376915.594089 \\
3 & 24.5 & 259.304375 & 57271 & -19.487611 & 17.2869583333 & 1229290.03354 & tu1670143.fits.fz & trilegal_photsys_file_j-2.dat & 0.04658868 & tab_mag_odfnew/tab_mag_lsst.dat & 0.04658868 & 4.594288 & 117.533165008 & 18051 & 387454.634903 \\
3 & 24.5 & 260.957167 & 118042 & -20.493167 & 17.3971444667 & 2533705.61261 & tu1677011.fits.fz & trilegal_photsys_file_j-4.dat & 0.04658868 & tab_mag_odfnew/tab_mag_lsst.dat & 0.04658868 & 3.90122 & 130.490627692 & 23718 & 509093.625318 \\
3 & 24.5 & 262.64325 & 15821